<center><h1>
Term Project – Draft 1
Imhoff, Tyler
SWENT 545 
</h1></center>


## I.	Introduction 

Members of the Department of Arts at the College of Arts and Sciences are currently tasked with addressing a few issues the university is facing. There are a few issues that must be addressed within the department. Enrollment has been stagnant over the last five years. There has been a change in the demographics and interests of the students. Shifts in disciplinary approaches. To address these issues, the Department of Arts is planning to reduce the number of credit hours from 39 to 33, to reduce the number of offered electives, and any elective courses are to be taken as a defined concentration area. 
	By reducing the number of electives and overall credit hours required, the department can cut costs and offer a more streamlined degree offering. There are currently a lot of courses that are being offered so cutting down on the options may help to reduce variations in student selections while continuing to provide a competitive degree program. 
	Before removing electives and other courses that are being offered, it should be determined which classes aren’t typically taken or don’t pair well together. We can utilize various data mining techniques to find correlations between different courses. Courses that have some correlation may provide information as to how new course concentrations may be applicable to the students. 


## II.	Description the dataset and initial preprocessing if any

The original dataset is a csv file that has 4918 rows and 3 columns. The 3 columns are students name, semester, and course name. There are 1245 duplicate rows in the dataset that will be removed along with 3 rows containing null values for course name. Any rows that list courses that are not relevant or not listed in the current course offerings are removed from the dataset. Additionally, student names that have misspellings or inconsistencies will be removed as well. 


In [109]:
from itertools import permutations
import collections
import pandas as pd
from collections import Counter

minimum_support = 0.2

pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

csv = pd.read_csv('Projectdata.csv')
courses_csv = pd.read_csv('ProjectData Courses.txt', sep='\t')

print(csv.head())
print(csv.info())

  Student name semester new                  coursename
0    Bill Mumy    Fall 2004     BEHAVIORAL PHARMACOLOGY
1    Bill Mumy    Fall 2000     AMERICAN FOREIGN POLICY
2    Bill Mumy    Fall 2003       DRUGS, BRAIN AND MIND
3    Bill Mumy    Fall 2005  Environmental Case Studies
4    Bill Mumy    Fall 2000     COMPUTER LINEAR ALGEBRA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4917 entries, 0 to 4916
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Student name  4917 non-null   object
 1   semester new  4917 non-null   object
 2   coursename    4914 non-null   object
dtypes: object(3)
memory usage: 115.4+ KB
None


In [110]:
# Remove duplicates and remove missing/null values 
duplicates = csv[csv.duplicated()]
null_rows = csv[csv.isnull().any(axis=1)]

print(duplicates)
print(null_rows)

csv = csv.dropna()
csv = csv.drop_duplicates()


                  Student name semester new                                         coursename
7                    Bill Mumy  Spring 2003                           CONTEMP ART - since 1945
8                    Bill Mumy  Spring 2003                           CONTEMP ART - since 1945
10                   Bill Mumy    Fall 2002                               ANALYTICAL MECHANICS
11                   Bill Mumy    Fall 2002                               ANALYTICAL MECHANICS
13                   Bill Mumy  Spring 2005                           CONTEMPORARY AFRICAN ART
14                   Bill Mumy  Spring 2005                           CONTEMPORARY AFRICAN ART
16                   Bill Mumy    Fall 2004                               CEL BIO BIOCHEMISTRY
17                   Bill Mumy    Fall 2004                               CEL BIO BIOCHEMISTRY
19                   Bill Mumy  Spring 2003                      CELL BIOLOGY and BIOCHEMISTRY
20                   Bill Mumy  Spring 2003       

In [111]:
# Export clean data to csv
csv.to_csv('noDupsNoNa_Projectdata.csv', index=False)
# Return data to dataframe
df = pd.read_csv('noDupsNoNa_Projectdata.csv')
print(df.info())

csv_file = 'ProjectData Courses.txt'
courses_df = pd.read_csv(csv_file, sep='\t', skiprows=1)
print(courses_df['Course Name'])
course_names_list = courses_df['Course Name'].tolist()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3669 entries, 0 to 3668
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Student name  3669 non-null   object
 1   semester new  3669 non-null   object
 2   coursename    3669 non-null   object
dtypes: object(3)
memory usage: 86.1+ KB
None
0       EXPERIMENTAL WRITING SEM: The Ecology of Poetry
1                                  ART: ancient to 1945
2                              ENVIRONMENTAL SYSTEMS II
3                               COMPUTER LINEAR ALGEBRA
4                                  ANALYTICAL MECHANICS
5                                        A WORLD AT WAR
6                               BEHAVIORAL PHARMACOLOGY
7                              CONTEMPORARY AFRICAN ART
8                              FOOD/FEAST ARCH OF TABLE
9                                 DEVIL'S PACT LIT/FILM
10                               AMERICAN SOCIAL POLICY
11                    

In [112]:
# Print all unique student names, for manual scanning of possible typos, and abbreviations
print("Unique student names:")
print(df['Student name'].count())
df['Student name'] = df['Student name'].str.title()
df['Student name'] = df['Student name'].str.strip()
unique_student_names = df['Student name'].unique()
print(unique_student_names)
print(df['Student name'].nunique())



Unique student names:
3669
['Bill Mumy' 'Chris Evert' 'Ruth Bader Ginsburg' 'Geraldine Ferraro'
 'Cch Pounder' 'Kitty Kelley' 'Peter Robinson' 'Laura Lippman'
 'Dom Deluise' 'Sally Field' 'Clint Black' 'Julie Christie 1995'
 'Pat Boone' 'Ed Mcmahon' 'David Cohen' 'Bill Blass' 'Lucille Ball'
 'H. Norman Schwarzkopf' 'Jan Burke' 'Bryant Gumbel' 'F. Lee Bailey'
 'Steven Spielberg' 'Gary Remal' 'Gene Tierney (Lee)' 'Kofi A. Annan'
 'Matthew Modine' 'Whoopi Goldberg' 'John Tesh' 'John Paul Stevens'
 'Laurie Metcalf' 'Ellen Burstyn' 'Patrick Stewart' 'Joel Grey'
 'Jerry Lewis' 'Willard Scott' 'Glenda Jackson' 'Mary Higgins Clark'
 'Tjessica Tandy' 'Sean Faircloth,' 'State Representative'
 'Janet Evanovich' 'Howard Keel' 'Kewing Kauffman' 'Jackie Collins'
 'Molly Ivins' 'Margaret Geller' 'Michael Ansara' 'Billy Collins'
 'Rhonda Fleming Mann' 'Jane Russell' 'Muhammad Ali' 'Philip Knight'
 'Victor K. Kiam Ii' 'Ed Bradley' 'Pat Carroll' 'Ppatti Page'
 'Reba Mcentire' 'Gov. Mario Cuomo' 'Shari L

In [113]:
# Correct inconsistencies in course names
print("Unique course names:")
print(df['coursename'].nunique())

# Courses contained in course_names_list are returned true. Those rows are kept. 
course_names_list_upper = [item.upper() for item in course_names_list]
df = df[df['coursename'].str.upper().isin(course_names_list_upper)]
        

print(courses_df['Course Name'].head(10))
print(len(course_names_list))


courses_joined = []

for c in df['coursename'].str.upper():
    if c in course_names_list:
        courses_joined.append(c)

print("Courses not offered by the program removed.\n")
print("Number of courses in data offered by the department: ", len(courses_joined))

Unique course names:
169
0    EXPERIMENTAL WRITING SEM: The Ecology of Poetry
1                               ART: ancient to 1945
2                           ENVIRONMENTAL SYSTEMS II
3                            COMPUTER LINEAR ALGEBRA
4                               ANALYTICAL MECHANICS
5                                     A WORLD AT WAR
6                            BEHAVIORAL PHARMACOLOGY
7                           CONTEMPORARY AFRICAN ART
8                           FOOD/FEAST ARCH OF TABLE
9                              DEVIL'S PACT LIT/FILM
Name: Course Name, dtype: object
42
Courses not offered by the program removed.

Number of courses in data offered by the department:  1593


In [114]:
student_schedule = df.groupby('Student name')['coursename'].apply(list) ## Make sure the data is clean
student_schedule.index = [f"{i}" for i in student_schedule.index]
pd.set_option('display.width', 1000)
print(student_schedule.head())

Abella Abzug    [BEHAVIORAL PHARMACOLOGY, FOOD/FEAST ARCH OF T...
Al Gore         [BEHAVIORAL PHARMACOLOGY, ANALYTICAL MECHANICS...
Al Hirt         [BEHAVIORAL PHARMACOLOGY, ENVIRONMENTAL SYSTEM...
Al Roker        [BEHAVIORAL PHARMACOLOGY, FOOD/FEAST ARCH OF T...
Alan Bates      [BEHAVIORAL PHARMACOLOGY, FOOD/FEAST ARCH OF T...
Name: coursename, dtype: object


In [115]:
student_schedule = list(student_schedule)
# Define the set of groceries
flattened = [i for t in student_schedule for i in t]
course_counts = {}
for item in flattened:
    if item in course_counts:
        course_counts[item] += 1
    else:
        course_counts[item] = 1

sorted_itemCounts = dict(sorted(course_counts.items(), key=lambda item: item[1],
                                reverse=True))
df_course_counts = pd.DataFrame(list(sorted_itemCounts.items()), columns =
    ['Items', 'Count'])
display(df_course_counts)

,Items,Count
0,A WORLD AT WAR,269
1,BEHAVIORAL PHARMACOLOGY,259
2,ANALYTICAL MECHANICS,256
3,CONTEMPORARY AFRICAN ART,217
4,FOOD/FEAST ARCH OF TABLE,112
5,EXPERIMENTAL WRITING SEM: The Ecology of Poetry,92
6,ELEMENTARY ARABIC II,64
7,AMERICAN HEALTH POLICY,51
8,CONTEMPORARY POL.THOUGHT,49
9,FRESHWATER ECOLOGY,46


## III. Model Generation

Based on the data format and what we're trying to accomplish, an apriori algorithm is going to be helpful in generating frequent itemsets. With this model, we can determine which courses are often taken together the most based on the student's schedules from the last few years. 

In [116]:
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder


courses_flattened = list(set(flattened))
# Generate all possible rules from the groceries list
rules = list(permutations(courses_flattened, 2))
print(f"Numer of rules: {len(rules)}")
# Instantiate transaction encoder and identify unique items in transactions
encoder = TransactionEncoder().fit(student_schedule)
onehot = encoder.transform(student_schedule)
onehot = pd.DataFrame(onehot, columns=encoder.columns_)
onehot.index = [f"Student {i+1}" for i in onehot.index]
display(onehot)

Numer of rules: 1190


,19TH-CENTURY BRITISH LITERATURE,20th Century Russian Literature: Fiction and Reality,A WORLD AT WAR,AESTHETICS,AFRICAN-AMERICAN LIT: AFRICAN-AMER LIT:CHANGE,AMERICAN HEALTH POLICY,AMERICAN SOUTH 1861-PRES,ANALYTICAL MECHANICS,ANALYZING THE POL WORLD,ART AND RELIGION,ART: ancient to 1945,BECOMING HUMAN,BEHAVIORAL PHARMACOLOGY,BRITISH POETRY 1660-1914,Business German: A Micro Perspective,CELL. BIOL. & BIOCHEM.,COMM and THE PRESIDENCY,COMMUNICATIONS INTERNSHP,COMPARATIVE POLITICS,COMPUTER LINEAR ALGEBRA,CONTEMP ART - 1945 to PRESENT,CONTEMPORARY AFRICAN ART,CONTEMPORARY POL.THOUGHT,DEVIL'S PACT LIT/FILM,ELEMENTARY ARABIC II,ELEMENTARY GERMAN 1,ENVIRONMENTAL SYSTEMS II,EUROPE IN A WIDER WORLD,EVIDENCED BASED CRIME AND JUSTICE POLICY,EXPERIMENTAL WRITING SEM: The Ecology of Poetry,Environmental Studies Research Seminar Junior Level,FOOD/FEAST ARCH OF TABLE,FRANCE & THE EUROP.UNION,FRESHWATER ECOLOGY,French Thought Since 1945
Student 1,False,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False
Student 2,False,True,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False
Student 3,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False
Student 4,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False
Student 5,False,False,True,False,False,False,False,True,False,False,False,True,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False
Student 6,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
Student 7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
Student 8,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
Student 9,False,True,True,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False
Student 10,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False


In [117]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = (apriori(onehot, min_support=minimum_support, use_colnames=True))
display(frequent_itemsets)

,support,itemsets
0,0.656863,(A WORLD AT WAR)
1,0.627451,(ANALYTICAL MECHANICS)
2,0.634804,(BEHAVIORAL PHARMACOLOGY)
3,0.531863,(CONTEMPORARY AFRICAN ART)
4,0.225490,(EXPERIMENTAL WRITING SEM: The Ecology of Poetry)
5,0.274510,(FOOD/FEAST ARCH OF TABLE)
6,0.495098,"(ANALYTICAL MECHANICS, A WORLD AT WAR)"
7,0.524510,"(BEHAVIORAL PHARMACOLOGY, A WORLD AT WAR)"
8,0.375000,"(CONTEMPORARY AFRICAN ART, A WORLD AT WAR)"
9,0.208333,"(FOOD/FEAST ARCH OF TABLE, A WORLD AT WAR)"


In [118]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)
display(rules)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ANALYTICAL MECHANICS),(A WORLD AT WAR),0.627451,0.656863,0.495098,0.789062,1.201259,0.082949,1.626725,0.449713
1,(A WORLD AT WAR),(ANALYTICAL MECHANICS),0.656863,0.627451,0.495098,0.753731,1.201259,0.082949,1.512775,0.488260
2,(BEHAVIORAL PHARMACOLOGY),(A WORLD AT WAR),0.634804,0.656863,0.524510,0.826255,1.257880,0.107531,1.974946,0.561375
3,(A WORLD AT WAR),(BEHAVIORAL PHARMACOLOGY),0.656863,0.634804,0.524510,0.798507,1.257880,0.107531,1.812455,0.597463
4,(CONTEMPORARY AFRICAN ART),(A WORLD AT WAR),0.531863,0.656863,0.375000,0.705069,1.073389,0.025639,1.163450,0.146049
5,(A WORLD AT WAR),(CONTEMPORARY AFRICAN ART),0.656863,0.531863,0.375000,0.570896,1.073389,0.025639,1.090963,0.199253
6,(FOOD/FEAST ARCH OF TABLE),(A WORLD AT WAR),0.274510,0.656863,0.208333,0.758929,1.155384,0.028018,1.423384,0.185374
7,(A WORLD AT WAR),(FOOD/FEAST ARCH OF TABLE),0.656863,0.274510,0.208333,0.317164,1.155384,0.028018,1.062467,0.391933
8,(BEHAVIORAL PHARMACOLOGY),(ANALYTICAL MECHANICS),0.634804,0.627451,0.519608,0.818533,1.304537,0.121300,2.052983,0.639230
9,(ANALYTICAL MECHANICS),(BEHAVIORAL PHARMACOLOGY),0.627451,0.634804,0.519608,0.828125,1.304537,0.121300,2.124777,0.626614
